<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#1.-Model-evaluation" data-toc-modified-id="1.-Model-evaluation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Model evaluation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#☑️-Check-your-code" data-toc-modified-id="☑️-Check-your-code-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>☑️ Check your code</a></span></li></ul></li></ul></li><li><span><a href="#2.-In-depth-diagnosis" data-toc-modified-id="2.-In-depth-diagnosis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. In depth diagnosis</a></span></li><li><span><a href="#3.-Reduced-training-size" data-toc-modified-id="3.-Reduced-training-size-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3. Reduced training size</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#☑️-Check-your-code" data-toc-modified-id="☑️-Check-your-code-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>☑️ Check your code</a></span></li></ul></li></ul></li><li><span><a href="#4.-Predictions" data-toc-modified-id="4.-Predictions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>4. Predictions</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#☑️-Check-your-code" data-toc-modified-id="☑️-Check-your-code-5.0.1"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span>☑️ Check your code</a></span></li></ul></li></ul></li><li><span><a href="#🏁" data-toc-modified-id="🏁-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>🏁</a></span></li></ul></div>

# Logistic Regression

🎯 Up until now, you have been modelling a regression task. This time, you are going to be modelling a classification task, on the famous Titanic dataset.

👇 Load the `titanic.csv` dataset into this notebook as a pandas dataframe, and display its first 5 rows.

<details>
    <summary>💡Hint</summary>
    The dataset is located in a parent directory, you should use <code>../</code> in order to find it.
</details>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/titanic.csv")
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

ℹ️ You can download a detailed description of the dataset [here](https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_Titanic_dataset_description.txt). Make sure to refer to it throughout the day.

# 1. Model evaluation

👇 We want to predict whether a passenger survived (`Survived`) the Titanic disaster, according to the following features:

- The class in which she/he was travelling (`Pclass`)
- The number of siblings/spouses he had onboard (`SibSp`)
- The number of parents/children he had onboard (`Parch`)
- The fare he paid for the ticket (`Fare`)

❓How accurate would a LogisticRegression model be at such a task? Save the accuracy under variable name `accuracy`.

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

X = df[["Pclass", "SibSp", "Parch", "Fare"]]
y = df["Survived"]

log_model = LogisticRegression()
cv = cross_validate(X=X, y=y, estimator=log_model, cv=5, scoring=["accuracy"])
accuracy = cv["test_accuracy"].mean()
accuracy

0.6835791852363318

In [4]:
# base accuracy
(df["Survived"].value_counts()/len(df))*100

0    61.616162
1    38.383838
Name: Survived, dtype: float64

### ☑️ Check your code

In [5]:
from nbresult import ChallengeResult

result = ChallengeResult('accuracy',
                         accuracy = accuracy
)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/Moe/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/Moe/code/moe221/data-challenges-moe/05-ML/01-Fundamentals-of-Machine-Learning/04-Logistic-Regression
plugins: dash-2.0.0, anyio-3.3.2, cov-3.0.0
collecting ... collected 1 item

tests/test_accuracy.py::TestAccuracy::test_accuracy PASSED               [100%]

============================== 1 passed in 0.12s ===============================


💯 You can commit your code:

git add tests/accuracy.pickle

git commit -m 'Completed accuracy step'

git push origin master


# 2. In depth diagnosis

❓We've evaluated the accuracy of the model at predicting, but we want to investigate its performance in more depth. Is the model overfitting? underfitting? How many learning examples does it need to learn optimally on the given features?

In [15]:
train_sizes = [*range(1, int(0.5*df.shape[0]), 50)]
train_sizes

[1, 51, 101, 151, 201, 251, 301, 351, 401]

In [32]:
from sklearn.model_selection import learning_curve

train_sizes = [*range(1, int(0.5*df.shape[0]), 50)]

lc = learning_curve(X=X, y=y, estimator=log_model, train_sizes=np.linspace(0.001, 1.0, 80), cv=5)

lc_train = lc[1].mean(axis=1)
lc_test = lc[2].mean(axis=1)
lc_sizes = lc[0]

fig = plt.figure(figsize=(10, 5))

ax = fig.gca()

sns.lineplot(x=lc_sizes, y=lc_test, ax=ax, label="test")
sns.lineplot(x=lc_sizes, y=lc_train, ax=ax, label="train")
lc

(array([  1,   9,  18,  27,  36,  45,  54,  63,  72,  81,  90,  99, 108,
        117, 126, 135, 144, 153, 162, 171, 180, 189, 198, 207, 216, 225,
        234, 243, 252, 261, 270, 279, 288, 297, 306, 315, 324, 333, 342,
        351, 360, 369, 378, 387, 396, 405, 414, 423, 432, 441, 450, 459,
        468, 477, 486, 495, 504, 513, 522, 531, 540, 549, 558, 567, 576,
        585, 594, 603, 612, 621, 630, 639, 648, 657, 666, 675, 684, 693,
        702, 712]),
 array([[       nan,        nan,        nan,        nan,        nan],
        [       nan, 0.77777778, 0.77777778, 0.77777778, 0.77777778],
        [       nan, 0.77777778, 0.77777778, 0.77777778, 0.77777778],
        [0.96296296, 0.74074074, 0.74074074, 0.74074074, 0.74074074],
        [0.86111111, 0.58333333, 0.58333333, 0.58333333, 0.58333333],
        [0.82222222, 0.6       , 0.6       , 0.6       , 0.6       ],
        [0.83333333, 0.64814815, 0.64814815, 0.64814815, 0.64814815],
        [0.82539683, 0.65079365, 0.65079365, 0.65079


<details>
<summary> ℹ️ Once you have completed your diagnosis, unfold this cell </summary>   
    
<br/>
You should have plotted the learning curves.

👉 The curves should have converged:
- The model is not overfitting (no variance): it generalizes well.

👉 The test score stops increasing around 500 training observations:
- The model does not need more than 500 training observations to perform optimally on the given features.
    
</details>


# 3. Reduced training size

👇 Evaluate a model on the training size suggested by the curves. Are you able to maintain a similar accuracy? Save the new accuracy under variable name `reduced_training_accuracy`

In [37]:
X = df.head(500)[["Pclass", "SibSp", "Parch", "Fare"]]
y = df.head(500)["Survived"]

log_model = LogisticRegression()
cv = cross_validate(X=X, y=y, estimator=log_model, cv=5, scoring=["accuracy"])
reduced_training_accuracy = cv["test_accuracy"].mean()
reduced_training_accuracy

0.65

ℹ️ The accuracy should be the nearly the same, with a reduced computational expense.

### ☑️ Check your code

In [38]:
from nbresult import ChallengeResult

result = ChallengeResult('reduced_accuracy',
                         accuracy = reduced_training_accuracy
)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/Moe/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/Moe/code/moe221/data-challenges-moe/05-ML/01-Fundamentals-of-Machine-Learning/04-Logistic-Regression
plugins: dash-2.0.0, anyio-3.3.2, cov-3.0.0
collecting ... collected 1 item

tests/test_reduced_accuracy.py::TestReduced_accuracy::test_accuracy PASSED [100%]

============================== 1 passed in 0.14s ===============================


💯 You can commit your code:

git add tests/reduced_accuracy.pickle

git commit -m 'Completed reduced_accuracy step'

git push origin master


# 4. Predictions

❓ Did the following person survive the Titanic disaster?

- Travelled in class 1 (`Pclass`)
- Had no spouse or sibblings on board(`SibSp`)
- Had no parents or children on board (`Parch`)
- Paid 15 for his ticket (`Fare`)

⚠️ Make sure the model you use for predicting has the **least computational expense possible**. Save the prediction under variable name `prediction`.

In [41]:
log_model = LogisticRegression()
log_model.fit(X.values, y)
prediction = log_model.predict([[1, 0, 0, 15]])
prediction

array([1])

❓ What is the probability that the person survived? Compute your answer and save it under variable name `probability`.

In [45]:
probability = log_model.predict_proba([[1, 0, 0, 15]])[0][1]
probability

0.5133385200560083

### ☑️ Check your code

In [46]:
from nbresult import ChallengeResult

result = ChallengeResult('predictions',
                         prediction = prediction,
                         probability = probability
)

result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/Moe/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/Moe/code/moe221/data-challenges-moe/05-ML/01-Fundamentals-of-Machine-Learning/04-Logistic-Regression
plugins: dash-2.0.0, anyio-3.3.2, cov-3.0.0
collecting ... collected 3 items

tests/test_predictions.py::TestPredictions::test_prediction_is_1 PASSED  [ 33%]
tests/test_predictions.py::TestPredictions::test_probability PASSED      [ 66%]
tests/test_predictions.py::TestPredictions::test_probability_type PASSED [100%]

============================== 3 passed in 0.13s ===============================


💯 You can commit your code:

git add tests/predictions.pickle

git commit -m 'Completed predictions step'

git push origin master


# 🏁